<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/basics/pytorch_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorchで体験するニューラルネットワーク構築（Irisデータセット）

この演習では、**PyTorch (パイトーチ)** という、機械学習（特にディープラーニング）のためのライブラリを使用して、基本的なニューラルネットワークをゼロから構築する流れを学びます。

### 💬 PyTorchとは？

PyTorchは、Pythonで広く使われている機械学習ライブラリです。主な特徴は以下の通りです。

1.  **テンソル (Tensor) の計算:** NumPyの配列（ndarray）に似ていますが、GPU（画像処理装置）を使った高速な並列計算に最適化されています。ニューラルネットワークは大量の行列計算を行うため、GPUによる高速化が不可欠です。
2.  **自動微分機能:** ニューラルネットワークの学習（パラメータの最適化）には、微分計算が必要です。PyTorchは、複雑な計算グラフに対しても、その微分（勾配）を自動で計算する機能（`autograd`）を提供します。
3.  **柔軟なモデル構築:** モデルの定義や学習プロセスをPythonのコードで直感的に記述でき、研究者や開発者に人気があります。

この演習では、PyTorchの基本的な構成要素（層、損失関数、オプティマイザ）を自分で組み合わせて、シンプルなニューラルネットワーク（多層パーセプトロン）を構築していきます。

### 🧠 ニューラルネットワークの基本構造

ニューラルネットワークは、人間の脳の神経細胞（ニューロン）の仕組みを単純化して模倣した数学モデルです。データから複雑なパターンを学習することができます。

基本的なニューラルネットワークは、いくつかの「層 (Layer)」から構成されます。

1.  **入力層 (Input Layer):**
    * データ（特徴量）を最初に入力する層です。
    * Irisデータセットの場合、「ガクの長さ」「ガクの幅」「花弁の長さ」「花弁の幅」の4つの特徴量があるので、入力層のノード（丸い部分）数は4になります。

2.  **隠れ層 (Hidden Layer):**
    * 入力層と出力層の間にあり、目に見えない中間的な計算を行う層です。
    * この層のノード数や層の数（深さ）を増やすことで、モデルはより複雑なパターンを学習できるようになります。
    * ノード同士は「重み (Weight)」と呼ばれるパラメータで接続されており、学習とはこの「重み」を適切な値に調整していく作業を指します。

3.  **出力層 (Output Layer):**
    * モデルの最終的な計算結果（予測）を出力する層です。
    * Irisデータセットの場合、「Setosa」「Versicolor」「Virginica」の3つの品種に分類する問題なので、出力層のノード数は3になります。各ノードは、それぞれの品種である「確率」や「スコア」を出力します。

PyTorchを使えば、これらの層を定義し、それらを `forward` という関数でつなぎ合わせるだけで、ニューラルネットワークモデルを比較的簡単に記述できます。

---

### 📚 1. 準備: ライブラリのインポート

まず、演習に必要なライブラリをインポートします。
それぞれのライブラリの役割は以下の通りです。

* `torch`: PyTorch本体です。テンソルの計算や自動微分など、中心的な機能が含まれます。
* `torch.nn (Neural Network)`: ニューラルネットワークの層（`nn.Linear`など）、損失関数（`nn.CrossEntropyLoss`など）が定義されています。
* `torch.optim (Optimizer)`: パラメータを最適化するアルゴリズム（`optim.Adam`など）が含まれます。
* `torch.utils.data`: データを効率的に扱うための `DataLoader` や `TensorDataset` が含まれます。
* `sklearn`: scikit-learnという機械学習ライブラリです。今回はデータセットのロード、標準化、データ分割に使用します。
* `numpy`: Pythonで数値計算を行うための基本的なライブラリです。
* `matplotlib.pyplot`: グラフを描画するために使用します。
* `tqdm.auto`: 学習の進捗状況をプログレスバーで表示するために使用します。

In [ ]:
# torch をインポートしてください
# torch.nn というモジュールを nn という名前でインポートしてください

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# 乱数シードの固定（結果の再現性を高めるため）
# これにより、何度実行しても同じ結果が得られやすくなります
torch.manual_seed(42)
np.random.seed(42)

---

### 🔢 2. データの準備

#### データのロードと「標準化」
scikit-learnを使ってIrisデータセットをロードします。

次に、**標準化 (Standardization)** を行います。これは、各特徴量（ガクの長さ、幅など）の平均を0、分散を1に揃える処理です。
例えば、「ガクの長さ」は 4.3〜7.9 の範囲ですが、「ガクの幅」は 2.0〜4.4 の範囲です。このように特徴量ごとにスケール（値の範囲）が大きく異なると、学習が非効率になったり、特定の（値が大きい）特徴量に学習が偏ったりすることがあります。
標準化によってスケールを揃えることは、ニューラルネットワークの学習をスムーズに進めるための一般的な前処理です。

In [ ]:
# データをロード
iris = load_iris()
X = iris.data
y = iris.target

# データを標準化 (平均0, 分散1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"標準化前の平均: {X.mean(axis=0)}")
print(f"標準化後の平均: {X_scaled.mean(axis=0)}")
print(f"標準化後の標準偏差: {X_scaled.std(axis=0)}")

#### 「テンソル」への変換
PyTorchは、**Tensor（テンソル）** という専用のデータ形式で計算を行います。これはNumpyの配列（ndarray）に似ていますが、GPUでの高速計算に対応している点が大きな特徴です。
Numpyの配列をPyTorchのテンソルに変換します。

**【課題1】** `( XXXX )` を埋めて、Numpy配列 `X_scaled` と `y` をテンソルに変換してください。
* 特徴量 `X` は、小数点以下の数値を含むため `torch.FloatTensor` を使います。
* ラベル `y` は、0, 1, 2 といったクラス番号（整数）であり、後で使う損失関数 `CrossEntropyLoss` の要求仕様でもあるため `torch.LongTensor` を使います。

In [ ]:
# データをPyTorchテンソルに変換
# 課題1: ( XXXX ) を埋めてください
X_tensor = torch.FloatTensor( ( XXXX ) )
y_tensor = torch.LongTensor( ( XXXX ) )

print(f"特徴量テンソルの形状: {X_tensor.shape}")
print(f"ラベルテンソルの形状: {y_tensor.shape}")

#### データの分割
モデルの性能を正しく評価するため、データセットを3つに分割します。これは機械学習において非常に重要なステップです。

1.  **教師データ (Training Data):**
    * モデルの学習（パラメータの更新）に直接使用します。
    * モデルはこのデータを見て、入力（特徴量）と出力（正解ラベル）の関係を学びます。

2.  **検証データ (Validation Data):**
    * 学習中のモデルの性能を「監視」するために使用します。
    * モデルは**このデータを見て学習（重みの更新）をしません。**
    * 主な役割は、「過学習（Overfitting）」をチェックすることです。教師データでの性能は上がっているのに、検証データでの性能が下がり始めたら、モデルが教師データを「丸暗記」し始めている兆候（過学習）であり、学習を止めるべきタイミング（**Early Stopping**）だと判断できます。
    * また、学習率や隠れ層のノード数といった「ハイパーパラメータ」を調整する際にも、この検証データの性能を指標にします。

3.  **テストデータ (Test Data):**
    * 学習プロセス（学習・検証）では**一切使用しません。**
    * 学習とハイパーパラメータ調整がすべて完了した後、最終的に完成したモデルの「真の性能（汎化性能）」を評価するために、**一度だけ**使用します。
    * 検証データも調整に使ってしまった以上、モデルは（間接的に）検証データに最適化されている可能性があるため、完全に未知のデータであるテストデータでの評価が最も客観的な指標となります。

ここでは、`stratify=y_tensor` というオプションを指定しています。これは、元のデータセットのクラスの比率（Setosa, Versicolor, Virginicaが約1:1:1）が、分割後の教師データ、検証データ、テストデータのすべてにおいて保たれるようにする（層化サンプリング）ための指定です。

**【課題2】** `( XXXX )` を埋めて、データを分割してください。
* ヒント: 最初の分割では `X_tensor` と `y_tensor` を使います。2回目で、1回目の残り（`X_temp`, `y_temp`）をさらに半分にします。

In [ ]:
# データを分割 (教師 70%, 検証 15%, テスト 15%)

# 課題2: ( XXXX ) を埋めてください

# まず、教師データ(70%)と一時データ(30%)に分ける
X_train, X_temp, y_train, y_temp = train_test_split(
    ( XXXX ), ( XXXX ),
    test_size=0.3,
    random_state=42,
    stratify=y_tensor # ラベルの比率を保つ
)

# 次に、一時データ(30%)を検証データ(15%)とテストデータ(15%)に分ける
X_val, X_test, y_val, y_test = train_test_split(
    ( XXXX ), ( XXXX ),
    test_size=0.5,
    random_state=42,
    stratify=y_temp # ラベルの比率を保つ
)

print(f"教師データ数: {len(y_train)}")
print(f"検証データ数: {len(y_val)}")
print(f"テストデータ数: {len(y_test)}")

---

### 📦 3. データローダーの作成

`DataLoader` は、PyTorchでの学習を効率化するための仕組みです。主な役割は以下の2つです。

1.  **バッチ処理 (Batch Processing):**
    * データセット全体（例: 105個）を一度にモデルに入力するのではなく、「**バッチサイズ (Batch Size)**」という小さなまとまり（例: 16個ずつ）に分けて供給します。
    * これにより、GPUメモリの使用量を抑えつつ、学習を安定させる効果があります（一度に大量のデータで計算するより、少しずつ計算して頻繁にパラメータを更新する方が、効率的に最適解にたどり着きやすいため）。

2.  **シャッフル (Shuffle):**
    * 教師データ（`train_loader`）について、学習を1周（1エポック）するごとにデータの順序をランダムに並び替えます。
    * これにより、モデルがデータの「順序」を学習してしまう（例えば「Setosaの次にはVersicolorが来やすい」といった、本来学習すべきでないパターンを覚えてしまう）ことを防ぎます。

まず、特徴量（X）とラベル（y）をペアにした `TensorDataset` を作成し、それを `DataLoader` に渡します。

**【課題3】** `( XXXX )` を埋めて、各データセット（train, val, test）の `TensorDataset` を作成してください。

In [ ]:
# 課題3: ( XXXX ) を埋めてください

# 1. TensorDatasetの作成 (Xとyをペアにする)
train_dataset = TensorDataset( ( XXXX ), ( XXXX ) )
val_dataset = TensorDataset( ( XXXX ), ( XXXX ) )
test_dataset = TensorDataset( ( XXXX ), ( XXXX ) )

# 2. DataLoaderの作成
BATCH_SIZE = 16 # バッチサイズ（一度に学習するサンプル数）

# shuffle=True は教師データのみ（学習時に順序をランダム化するため）
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

---

### 🧠 4. モデル（ネットワーク）の定義

いよいよニューラルネットワークの「設計図」を作成します。
PyTorchでは `torch.nn.Module` を継承したクラスとして定義するのが標準的です。

* `__init__` (コンストラクタ):
    * ネットワークの「部品」（層）をあらかじめ定義しておく場所です。
    * `nn.Linear(A, B)` は、A個の入力ノードからB個の出力ノードへの**全結合層（Linear Layer）**を意味します。
* `forward` (順伝播):
    * データがネットワークをどのように流れるか（計算の順序）を定義する場所です。
    * `__init__` で定義した部品を、ここで順番に組み合わせていきます。

#### 活性化関数 (Activation Function) とは？
`nn.Linear` は「線形変換」（入力に重みを掛けて足し合わせる）しかできません。線形変換だけをいくら重ねても、結果は線形のままです。
しかし、現実世界のパターン（例: Irisの品種の境界線）は単純な直線（線形）では分けられないことがほとんどです。

そこで、層と層の間に「**活性化関数**」と呼ばれる「非線形 (Non-linear)」な関数を挟みます。これにより、モデルは複雑な曲線的な境界線を学習できるようになります。

今回は、代表的な活性化関数である `nn.ReLU()` (ランプ関数) を使用します。これは、入力が0以下なら0を、0より大きければその値をそのまま出力する、非常にシンプルな非線形関数です。

**【課題4】** `( XXXX )` を埋めて、入力層(4) → 隠れ層(10) → 出力層(3) のネットワークを完成させてください。

In [ ]:
class SimpleNeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        # 親クラス(nn.Module)の初期化を呼び出す
        super(SimpleNeuralNet, self).__init__()

        # 課題4 (1/3): ネットワークの層を定義
        # 第1層 (入力層 -> 隠れ層)
        # 入力ノード数: input_dim, 出力ノード数: hidden_dim
        self.fc1 = nn.Linear( ( XXXX ) , ( XXXX ) )

        # 活性化関数 (今回はReLUを使います)
        self.relu = nn.ReLU()

        # 第2層 (隠れ層 -> 出力層)
        # 入力ノード数: hidden_dim, 出力ノード数: output_dim
        self.fc2 = nn.Linear( ( XXXX ) , ( XXXX ) )

    def forward(self, x):

        # 課題4 (2/3): データが流れる順序を定義
        # x -> fc1 -> relu -> fc2 -> output

        # 第1層を通して...
        out = self. ( XXXX ) (x)
        # 活性化関数を通して...
        out = self. ( XXXX ) (out)
        # 第2層を通して...
        out = self. ( XXXX ) (out)

        return out

# --- モデルのパラメータ設定 ---
INPUT_DIM = 4      # 入力層のノード数（特徴量の数）
HIDDEN_DIM = 10    # 隠れ層のノード数（この値は自由に変更して試せます）
OUTPUT_DIM = 3     # 出力層のノード数（クラスの数）

# 課題4 (3/3): モデルのインスタンス（実体）を作成
model = SimpleNeuralNet( ( XXXX ) , ( XXXX ) , ( XXXX ) )

# モデルの構造を確認
print(model)

---

### 🎯 5. 損失関数とオプティマイザの選択

モデルの学習に必要な「学習方針」を決める、重要な構成要素を2つ選びます。

1.  **損失関数 (Loss Function / Criterion):**
    * モデルの予測と正解の「ズレ（誤差）」を数値化する関数です。
    * 学習の目標は、この**損失の値を最小にすること**です。
    * 今回は、多クラス分類問題で最も一般的に使われる「**クロスエントロピー損失**」 (`nn.CrossEntropyLoss`) を使用します。これは、モデルが予測した「確率分布（3クラスの確率）」と「正解（例: クラス2が正解）」が、どれだけ異なっているかを測る指標です。

2.  **オプティマイザ (Optimizer / 最適化手法):**
    * 損失を最小化するために、モデルのパラメータ（重み）を「どのように更新するか」を決めるアルゴリズムです。
    * 損失の値（誤差）に基づいて、各パラメータがどちらの方向に（プラスかマイナスか）、どれくらい動けば損失が減るか（＝勾配）を計算し、パラメータを少しずつ更新していきます。
    * 今回は「**Adam**」 (`optim.Adam`) という、効率的で高性能なため広く使われているアルゴリズムを使用します。

3.  **学習率 (Learning Rate):**
    * オプティマイザがパラメータを一度にどれくらいの「歩幅」で更新するかを決める値です。
    * 大きすぎると最適解を通り過ぎて学習が不安定になり、小さすぎると学習に時間がかかりすぎます。Adamは、この学習率をある程度自動で調整してくれる機能も持っています。

**【課題5】** `( XXXX )` を埋めて、損失関数とオプティマイザを設定してください。

In [ ]:
# 学習率 (Learning Rate)
LEARNING_RATE = 0.01

# 課題5: ( XXXX ) を埋めてください

# 損失関数 (Cross Entropy Loss)
# nn.CrossEntropyLoss は、内部でSoftmaxの計算も含むため、モデルの出力層にSoftmaxを入れなくてもよい
criterion = nn. ( XXXX ) ()

# オプティマイザ (Adam)
# model.parameters() で、モデル内の学習対象パラメータ（重み）をすべて渡します。
optimizer = optim. ( XXXX ) (model.parameters(), lr= ( XXXX ) )

---

### 🏋️ 6. 学習（トレーニング）の実行

いよいよモデルを学習させます。PyTorchでは、学習ループ（エポックごとの繰り返し）を自分で書く必要があります。

**エポック (Epoch)** とは、教師データセット全体を1周学習することです。

**学習の1ステップ（ミニバッチごと）:**
学習ループの中では、`train_loader` からバッチ（16個のデータ）を一つずつ取り出し、以下の処理を繰り返します。

1.  `optimizer.zero_grad()`: **勾配のリセット**。PyTorchは勾配を（意図的に）累積する仕様のため、各バッチの計算開始時に必ずリセットが必要です。
2.  `outputs = model(inputs)`: **順伝播 (Forward Propagation)**。データをモデルに入力し、予測（出力）を得ます。
3.  `loss = criterion(outputs, labels)`: **損失の計算**。予測と正解ラベルを比べ、損失（誤差）を計算します。
4.  `loss.backward()`: **誤差逆伝播 (Backpropagation)**。損失を各パラメータに逆伝播させ、勾配（損失を減らすためのパラメータの更新方向）を自動計算します。
5.  `optimizer.step()`: **パラメータの更新**。計算された勾配に基づき、オプティマイザがパラメータを更新します。

---
**モード切り替え:**
* `model.train()`: 学習モード。Dropout層など、学習時と評価時で挙動が異なる層を正しく動作させるために呼び出します。（今回は使いませんが、習慣として記述します）
* `model.eval()`: 評価モード。学習を行わない（Dropoutを無効化するなど）状態にします。検証データやテストデータを評価する前に呼び出します。

**勾配計算の停止:**
* `with torch.no_grad()`: 評価時（`val_loader` や `test_loader` を使う時）は、パラメータの更新も誤差逆伝播も不要です。このブロックで囲むことで、勾配計算をすべて停止し、余計な計算を省いてメモリ効率と速度を向上させます。

**【課題6】** `( XXXX )` を埋めて、学習ループの核となる5ステップを完成させてください。

In [ ]:
EPOCHS = 200      # 最大エポック数
PATIENCE = 10     # Early Stoppingのための我慢回数（この回数連続で改善がなければ停止）

# ログ（履歴）の保存用
history = {
    'train_loss': [],
    'val_loss': [],
    'train_acc': [],
    'val_acc': []
}

best_val_loss = float('inf') # 検証損失のベストスコアを無限大で初期化
trigger_times = 0 # 損失が改善しなかった回数のカウンター

# tqdmを使って進捗バーを表示
pbar = tqdm(range(EPOCHS), desc="Epochs")

for epoch in pbar:

    # --- 教師データでの学習 (Train) ---
    model.train() # 学習モードに設定
    total_train_loss = 0
    total_train_correct = 0

    for inputs, labels in train_loader:

        # 課題6 (1/5): 勾配をリセット
        optimizer. ( XXXX ) ()

        # 課題6 (2/5): 順伝播 (forward)
        outputs = ( XXXX ) (inputs)

        # 課題6 (3/5): 損失の計算
        loss = ( XXXX ) (outputs, labels)

        # 課題6 (4/5): 逆伝播 (backward)
        loss. ( XXXX ) ()

        # 課題6 (5/5): パラメータの更新
        optimizer. ( XXXX ) ()

        # --- ここから下は集計・評価処理 ---
        total_train_loss += loss.item() * inputs.size(0) # バッチごとの損失を累積
        _, predicted = torch.max(outputs.data, 1) # 最も確率の高いクラスを予測とする
        total_train_correct += (predicted == labels).sum().item() # 正解数を累積

    # エポックごとの平均損失と平均精度を計算
    avg_train_loss = total_train_loss / len(train_loader.dataset)
    avg_train_acc = total_train_correct / len(train_loader.dataset)
    history['train_loss'].append(avg_train_loss)
    history['train_acc'].append(avg_train_acc)

    # --- 検証データでの評価 (Validation) ---
    model.eval() # 評価モードに設定
    total_val_loss = 0
    total_val_correct = 0

    with torch.no_grad(): # 勾配計算を無効化
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_val_correct += (predicted == labels).sum().item()

    avg_val_loss = total_val_loss / len(val_loader.dataset)
    avg_val_acc = total_val_correct / len(val_loader.dataset)
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(avg_val_acc)

    # 進捗バーに情報を表示
    pbar.set_postfix({
        'TrainLoss': f'{avg_train_loss:.4f}',
        'ValLoss': f'{avg_val_loss:.4f}',
        'TrainAcc': f'{avg_train_acc:.4f}',
        'ValAcc': f'{avg_val_acc:.4f}'
    })

    # --- Early Stoppingの判定 ---
    if avg_val_loss < best_val_loss:
        # 検証損失が改善した場合
        best_val_loss = avg_val_loss
        trigger_times = 0 # カウンターをリセット
        # (オプション: 最も性能の良いモデルのパラメータを保存)
        # torch.save(model.state_dict(), 'best_model.pth')
    else:
        # 検証損失が改善しなかった場合
        trigger_times += 1 # カウンターを増やす
        if trigger_times >= PATIENCE:
            # 我慢回数を超えたら学習を停止
            print(f'\nEarly stopping at epoch {epoch+1}')
            pbar.close()
            break

if (epoch + 1) == EPOCHS:
    print('\nFinished Training (Reached max epochs)')

---

### 📊 7. 結果の可視化

学習の経過（損失と精度）をグラフで確認します。

* **損失 (Loss) グラフ:** 学習が進むにつれて、教師データ（Train Loss）と検証データ（Validation Loss）の両方が下がっていくのが理想です。
* **精度 (Accuracy) グラフ:** 逆に、両方の精度が上がっていくのが理想です。

#### 「過学習 (Overfitting)」の兆候
もし、`Train Loss` は下がり続けるのに `Validation Loss` が上昇し始めたり（グラフが「V」字に開く）、`Train Accuracy` が上がり続けるのに `Validation Accuracy` が横ばいや低下し始めたりした場合、それが**過学習**の兆候です。
モデルは教師データに過剰に適合（ほぼ丸暗記）してしまい、未知のデータ（検証データ）に対する性能が落ちていることを示します。
`Early Stopping` は、`Validation Loss` が上昇し始める（または改善しなくなる）時点で学習を止めることで、過学習を防ぐテクニックです。

In [ ]:
# グラフの描画
plt.figure(figsize=(12, 5))

# 損失 (Loss)
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# 精度 (Accuracy)
plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Accuracy')
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

---

### 💯 8. テストデータによる最終評価

最後に、学習にも検証（Early Stoppingの判断）にも使っていない、完全に未知の「テストデータ」 (`test_loader`) を使って、モデルの最終的な性能（汎化性能）を評価します。
これが、このモデルの「実力」を示す最も客観的な数値となります。

**【課題7】** `( XXXX )` を埋めて、テストデータで評価するコードを完成させてください。

* ヒント: 検証データの評価ループとほとんど同じですが、使用するデータローダーが異なります。

In [ ]:
model.eval() # 評価モード
total_test_correct = 0

# (オプション: Early Stoppingで保存したベストモデルをロードする場合)
# model.load_state_dict(torch.load('best_model.pth'))

with torch.no_grad():

    # 課題7 (1/3): 使用するデータローダー
    for inputs, labels in ( XXXX ):

        outputs = model(inputs)
        # 予測結果（最もスコアが高いクラスのインデックス）を取得
        _, predicted = torch.max(outputs.data, 1)

        # 課題7 (2/3): 正解(labels)と比較
        total_test_correct += ( ( XXXX ) == ( XXXX ) ).sum().item()

# 課題7 (3/3): テストデータセット全体で割る
test_accuracy = total_test_correct / len( ( XXXX ) .dataset)
print(f'Test Accuracy: {test_accuracy * 100:.2f} %')

---

### 🎉 9. まとめと発展

この演習では、PyTorchを使ってニューラルネットワークを構成する基本的な要素（データ準備、テンソル、データローダー、モデル定義、活性化関数、損失関数、オプティマイザ、学習ループ、評価）を学びました。
各ステップを自分で定義することで、ライブラリが内部で何を行っているかの理解が深まったかと思います。

#### 💡 発展課題
* `HIDDEN_DIM` (隠れ層のノード数) や `LEARNING_RATE` (学習率) を変えると、学習結果（グラフや最終精度）はどのように変わるか試してみましょう。
* `class SimpleNeuralNet` を変更して、隠れ層をもう1層増やしてみてください（`__init__` で `self.fc3` などを定義し、`forward` の流れを変更する必要があります）。精度は向上するでしょうか？